### Elo
Elo is a method of assigning a numeric value to how good a player is at some game. It is most commonly used in chess. A 400 elo difference means the stronger player should win 9 out of 10 games against the weaker player. 

We construct Elo based on match history. If a player wins a game, their elo will increase, and if they lose, their elo will decrease. This is scaled by the relative strength between the player and their opponent.

Expected Score:

$$E_A = \frac{1}{1 + 10^{(R_B - R_A)/400}}$$

$$E_B = 1 - E_A$$

Actual Score:
- Win: Actual Score = 1
- Lose: Actual Score = 0

The Elo update formula is:

New Rating = Old Rating + K × (Actual Score − Expected Score). 

K controls how fast or slow a player's rating changes after each game. Here we use an adaptive K value.

We also track elo by surface.

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_parquet("../../data/cleaned/atp_matches_cleaned.parquet")
print(df.columns)
print(df["surface"].unique())

Index(['surface', 'draw_size', 'tourney_level', 'tourney_date', 'id_a',
       'name_a', 'hand_a', 'ht_a', 'age_a', 'id_b', 'name_b', 'hand_b', 'ht_b',
       'age_b', 'score', 'best_of', 'round', 'minutes', 'ace_a', 'df_a',
       'svpt_a', '1stIn_a', '1stWon_a', '2ndWon_a', 'SvGms_a', 'bpSaved_a',
       'bpFaced_a', 'ace_b', 'df_b', 'svpt_b', '1stIn_b', '1stWon_b',
       '2ndWon_b', 'SvGms_b', 'bpSaved_b', 'bpFaced_b', 'rank_a',
       'rank_points_a', 'rank_b', 'rank_points_b', 'result'],
      dtype='object')
['Hard' 'Carpet' 'Clay' 'Grass']


In [4]:
import math
from datetime import datetime

# ----------------------------------------------------------------------
# Mapping from the surface string in the data to the dict key
# ----------------------------------------------------------------------
_SURFACE_KEY = {
    "Hard":   "elo_h",
    "Clay":   "elo_c",
    "Grass":  "elo_g",
}

surface_K_boost = {
    "elo_h": 1.0,   # Hard: no boost
    "elo_c": 1.5,   # Clay: +50%
    "elo_g": 2.5,   # Grass: +150%
}

def update_elo_dict(
    elos,
    match,
    default_elo=1500,
    K_new=64,
    K_base=20,
    tau=20,
):
    """
    Update global + surface-specific Elo ratings.

    match must contain:
        .id_a, .name_a, .id_b, .name_b
        .result          # 1 if A wins, 0 if B wins
        .tourney_date    # int YYYYMMDD
        .surface         # "Hard", "Clay", "Grass" or "Carpet"
    """
    # ------------------------------------------------------------------
    # 1. Initialise players if they are new
    # ------------------------------------------------------------------
    for pid, pname in [(match.id_a, match.name_a), (match.id_b, match.name_b)]:
        if pid not in elos:
            elos[pid] = {
                "name": pname,
                "elo":   default_elo,   # global
                "elo_h": default_elo,   # Hard
                "elo_c": default_elo,   # Clay
                "elo_g": default_elo,   # Grass
                "matches": 0,
                "last_game_played": None,
            }

    # ------------------------------------------------------------------
    # 2. Pull current global ratings (used for expectation)
    # ------------------------------------------------------------------
    old_a = elos[match.id_a]["elo"]
    old_b = elos[match.id_b]["elo"]

    # ------------------------------------------------------------------
    # 3. Expected win probabilities (global)
    # ------------------------------------------------------------------
    exp_a = 1 / (1 + 10 ** ((old_b - old_a) / 400.0))
    exp_b = 1 - exp_a

    # ------------------------------------------------------------------
    # 4. Dynamic K-factor (same for both players & surfaces)
    # ------------------------------------------------------------------
    K_a = K_base + (K_new - K_base) * math.exp(-elos[match.id_a]["matches"] / tau)
    K_b = K_base + (K_new - K_base) * math.exp(-elos[match.id_b]["matches"] / tau)

    # Grand slam boost
    if match.tourney_level == "G":
        K_a *= 1.8
        K_b *= 1.8

    # ------------------------------------------------------------------
    # 5. Result
    # ------------------------------------------------------------------
    res_a = match.result          # 1 = A wins
    res_b = 1 - res_a

    # ------------------------------------------------------------------
    # 6. Update **global** Elo
    # ------------------------------------------------------------------
    new_a_global = old_a + K_a * (res_a - exp_a)
    new_b_global = old_b + K_b * (res_b - exp_b)

    # ------------------------------------------------------------------
    # 7. Surface-specific update (only if the surface is known)
    # ------------------------------------------------------------------
    surface_key = _SURFACE_KEY.get(match.surface)
    if surface_key:                                 # Hard / Clay / Grass
        # pull the *current* surface rating
        old_a_surf = elos[match.id_a][surface_key]
        old_b_surf = elos[match.id_b][surface_key]

        # expectation **on that surface** (use surface ratings)
        exp_a_surf = 1 / (1 + 10 ** ((old_b_surf - old_a_surf) / 400.0))
        exp_b_surf = 1 - exp_a_surf

        # grass and clay more noisy, adjust K accordingly
        K_a = min(80, K_a * surface_K_boost[surface_key])
        K_b = min(80, K_b * surface_K_boost[surface_key])
            
        # new surface ratings
        new_a_surf = old_a_surf + K_a * (res_a - exp_a_surf)
        new_b_surf = old_b_surf + K_b * (res_b - exp_b_surf)

    # ------------------------------------------------------------------
    # 8. Write everything back
    # ------------------------------------------------------------------
    match_date = datetime.strptime(str(match.tourney_date), "%Y%m%d").date()

    elos[match.id_a].update({
        "elo":   new_a_global,
        "matches": elos[match.id_a]["matches"] + 1,
        "last_game_played": match_date,
    })
    elos[match.id_b].update({
        "elo":   new_b_global,
        "matches": elos[match.id_b]["matches"] + 1,
        "last_game_played": match_date,
    })

    # surface-specific write-back
    if surface_key:
        elos[match.id_a][surface_key] = new_a_surf
        elos[match.id_b][surface_key] = new_b_surf

    return match.id_a, match.id_b, match.name_a, match.name_b, new_a_global, new_b_global

We loop through all matches in the database and update the elos sequentially. We also apply an elo decay for players who have been inactive for a long time.

At the same time we can store each player's elo history for visualisation.

In [5]:
from collections import defaultdict
from datetime import datetime

_SURFACE_FIELDS = {"elo_h", "elo_c", "elo_g"}   # Hard, Clay, Grass

def apply_decay(
    elos,
    current_date,
    days_since_last_decay,
    half_life_days=120,
    mean_elo=1500,
    min_days_inactive=180,
):
    if not elos:
        return

    for pid, data in elos.items():
        last_played = data.get("last_game_played")
        if last_played is not None:
            days_inactive = (current_date - last_played).days
            if days_inactive >= min_days_inactive:
                decay_factor = 0.5 ** (days_since_last_decay / half_life_days)
                old_global = data["elo"]
                data["elo"] = mean_elo + decay_factor * (old_global - mean_elo)

        for surf_key in _SURFACE_FIELDS:
            if surf_key not in data:
                continue

            days_inactive = (current_date - last_played).days
                
            if days_inactive < min_days_inactive:
                continue

            decay_factor = 0.5 ** (days_since_last_decay / half_life_days)
            old_surf = data[surf_key]
            data[surf_key] = mean_elo + decay_factor * (old_surf - mean_elo)

elo_dict = {}
history = defaultdict(list)
last_date = None

# loop through all matches and find final elo
for row in df.itertuples(index=False):
    # decay elo
    current_date = datetime.strptime(str(row.tourney_date), "%Y%m%d").date()

    if last_date is not None:
        days = (current_date - last_date).days
        apply_decay(elo_dict, current_date, days)

    last_date = current_date
    
    id_a, id_b, name_a, name_b, elo_a, elo_b = update_elo_dict(elo_dict, row)
    history[id_a].append({"date": current_date, "elo": elo_a})
    history[id_b].append({"date": current_date, "elo": elo_b})

# package results into a dataframe
elo_df = pd.DataFrame(
    {
        "id": pid,
        "name": data["name"],
        "elo": data["elo"],
        "elo_c": data["elo_c"],
        "elo_g": data["elo_g"],
        "elo_h": data["elo_h"],
        "matches": data["matches"],
    }
    for pid, data in elo_dict.items()
)

KeyboardInterrupt: 

We can now visualise the elo dataframe we have just created. 

Leaderboards of the various elos:

In [ ]:
def find_player_by_name(elo_df, player_name):
    rows = elo_df.loc[elo_df["name"] == player_name]
    if rows.empty:
        raise KeyError(f"Player name {player_name} not found")
    return rows.iloc[0]

def display_top(column, k=20):
    # print the top 10 best players currently by elo
    elo_df_sorted = elo_df.sort_values(column, ascending=False)

    print("Top " + str(k) + " by " + column + ":")
    rank = 1
    for idx, row in elo_df_sorted.head(k).iterrows():
        print(rank, row["id"], row["name"], row[column])
        rank += 1
    print()

display_top("elo")
display_top("elo_g")
display_top("elo_h")
display_top("elo_c")

Top 20 by elo:
1 206173 Jannik Sinner 2044.8058944768213
2 104925 Novak Djokovic 1914.8094515789617
3 207989 Carlos Alcaraz 1884.4261119659614
4 100644 Alexander Zverev 1835.6793800739267
5 106421 Daniil Medvedev 1821.0206638462864
6 126203 Taylor Fritz 1817.091244489809
7 207733 Jack Draper 1772.6116337419855
8 105777 Grigor Dimitrov 1759.4062341516958
9 200282 Alex De Minaur 1757.4137022954149
10 126205 Tommy Paul 1730.6653611626452
11 208029 Holger Rune 1718.9506440508721
12 128034 Hubert Hurkacz 1697.0458513979015
13 111575 Karen Khachanov 1691.8590427380627
14 210097 Ben Shelton 1684.0076598544367
15 200005 Ugo Humbert 1683.0836129305096
16 126774 Stefanos Tsitsipas 1681.76201748257
17 126094 Andrey Rublev 1677.8141021335894
18 200624 Sebastian Korda 1673.7278191138216
19 126207 Frances Tiafoe 1672.492736956115
20 207830 Tomas Machac 1670.2080067424351

Top 20 by elo_g:
1 207989 Carlos Alcaraz 1995.5154496643622
2 104925 Novak Djokovic 1890.3825808680729
3 106421 Daniil Medvedev 1

We can also plot a player's elo over time and matches.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from datetime import datetime
from typing import List, Optional, Dict, Any

# Helper: turn the raw `history` dict into a tidy DataFrame
def _build_history_df(
    history: Dict[Any, List[Dict[str, Any]]],
    elo_dict: Dict[Any, Dict[str, Any]]
) -> pd.DataFrame:
    """Convert history + elo_dict → DataFrame with columns: id, name, date, elo."""
    records = []
    for pid, points in history.items():
        name = elo_dict[pid]["name"]
        for pt in points:
            records.append({
                "id": pid,
                "name": name,
                "date": pt["date"],
                "elo": pt["elo"]
            })
    df = pd.DataFrame(records)
    df = df.sort_values(["id", "date"]).reset_index(drop=True)
    return df


def plot_by_date(
    history: Dict[Any, List[Dict[str, Any]]],
    elo_dict: Dict[Any, Dict[str, Any]],
    column: str = "elo",
    player_ids: Optional[List[Any]] = None,
    title: str = "",
    figsize: tuple = (12, 6),
    marker_size: int = 3,
    line_width: float = 1.2,
    date_interval_months: int = 2,
    save_path: Optional[str] = None
) -> None:
    """
    Plot Elo over calendar time.

    Parameters
    ----------
    history : dict
        Your `history` dict from the match loop.
    elo_dict : dict
        Your final `elo_dict` (must contain `"name"` for each player).
    player_ids : list, optional
        IDs to plot. If None → top 5 by final Elo.
    title : str
        Plot title.
    figsize : tuple
        Figure size.
    marker_size, line_width : int/float
        Matplotlib styling.
    date_interval_months : int
        X-axis major tick spacing.
    save_path : str, optional
        If given, save PNG (300 dpi).
    """
    df = _build_history_df(history, elo_dict)

    # ----- choose players -----
    if player_ids is None:
        # top 5 by final Elo
        final_elos = {pid: data[column] for pid, data in elo_dict.items()}
        player_ids = sorted(final_elos, key=final_elos.get, reverse=True)[:5]

    df_plot = df[df["id"].isin(player_ids)].copy()

    # ----- plot -----
    plt.figure(figsize=figsize)
    for pid in player_ids:
        sub = df_plot[df_plot["id"] == pid]
        name = sub["name"].iloc[0]
        plt.plot(
            sub["date"], sub[column],
            marker='o', markersize=marker_size, linewidth=line_width,
            label=f"{name} (id:{pid})"
        )

    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.YearLocator())           # one tick per year
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))   # format as "2020"
    
    plt.gcf().autofmt_xdate()

    plt.title(title, fontsize=14)
    plt.xlabel("Date")
    plt.ylabel(column)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()


def plot_by_match(
    history: Dict[Any, List[Dict[str, Any]]],
    elo_dict: Dict[Any, Dict[str, Any]],
    column: str = "elo",
    player_ids: Optional[List[Any]] = None,
    title: str = "",
    figsize: tuple = (12, 6),
    marker_size: int = 3,
    line_width: float = 1.2,
) -> None:
    """
    Plot Elo against the number of matches each player has played.

    Parameters
    ----------
    Same as `plot_elo_by_date`, except no date formatting.
    """
    df = _build_history_df(history, elo_dict)

    # add match counter
    df["match_num"] = df.groupby("id").cumcount() + 1

    if player_ids is None:
        final_elos = {pid: data[column] for pid, data in elo_dict.items()}
        player_ids = sorted(final_elos, key=final_elos.get, reverse=True)[:5]

    df_plot = df[df["id"].isin(player_ids)].copy()

    plt.figure(figsize=figsize)
    for pid in player_ids:
        sub = df_plot[df_plot["id"] == pid]
        name = sub["name"].iloc[0]
        plt.plot(
            sub["match_num"], sub[column],
            marker='o', markersize=marker_size, linewidth=line_width,
            label=f"{name} (id:{pid})"
        )

    plt.title(title, fontsize=14)
    plt.xlabel("Match Number (career)")
    plt.ylabel(column)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

    plt.show()

In [ ]:
plot_by_date(history, elo_dict)

In [ ]:
plot_by_match(history, elo_dict)